# Challenger Space Shuttle Disaster

In this notebook we'll model the probability of Space Shuttle O-ring damage as a function of temperature, and estimate the probability of failure of the Challenger. 

This example is adapted from Davidson-Pilon, Cameron, *Bayesian Methods for Hackers: Probabilistic Programming and Bayesian Inference*, Addison Wesley, 2016, which is availabe in Jupyter Notebook form on github. 

Note that in the following discussion plotting details have been replaced by the resulting plot for the sake of clarity; please see the original Jupyter Notebook for plotting details.

On January 28, 1986, the twenty-fifth flight of the U.S. space shuttle program ended in disaster when one of the rocket boosters of the Shuttle Challenger exploded shortly after lift-off, killing all seven crew members. The presidential commission on the accident concluded that it was caused by the failure of an O-ring in a field joint on the rocket booster, and that this failure was due to a faulty design that made the O-ring unacceptably sensitive to a number of factors including outside temperature.

The weather was unusually cold (31F) on the morning of launch (the previous coldest launch was at 53F), and the Morton Thiokol engineers recommended delaying the launch until temperatures rose. The engineers did everything in their power to explain the risks, but were overridden by management. If the 2019 *Challenger Disaster* movie was accurate, management asked the engineers to __"quantify your concerns"__.

**One can only wonder if the outcome might have been different had the engineers had 2019 computing power and tools like PyMC3 available!**  We'll attempt to quantify the risk in terms that might have convinced the managers.

### The SRB Field Joint with O-Rings

![title](img/SRBJointLegend.png)
From https://en.wikipedia.org/wiki/Space_Shuttle_Challenger_disaster

### Importing the required libraries

In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pymc3 as pm
import theano.tensor as tt
import arviz as az

### O-Ring Damage Data
The boosters were routinely recovered by parachuting into the ocean. Of the previous 24 flights, data were available on failures of O-rings on 23, (one was lost at sea), and these data were discussed on the evening preceding the Challenger launch. The engineers recommended postponing the launch, which was to take place at a temperature of 31F (the coldest previous launch was at 53F). Managers however apparently focused only on damage incidents, and saw no apparent correlation between damage and temperature, and overruled the engineers. The launch proceeded with disastrous consequences.

The following figure shows observed damage on Space Shuttle flights prior to Challenger:
![title](img/o_ring_data.png)

In [2]:
challenger_data = np.genfromtxt("data/challenger_data.csv", skip_header=1,
                                usecols=[1, 2], missing_values="NA",
                                delimiter=",")
#drop the NA values
challenger_data = challenger_data[~np.isnan(challenger_data[:, 1])]
print(challenger_data[:10].T)  # <== print first 10 pairs of data

[[66. 70. 69. 68. 67. 72. 73. 70. 57. 63.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.  1.]]


### Quantifying the Engineers' Concerns

It looks clear that *the probability* of damage incidents occurring increases as the outside temperature decreases. We are interested in modeling the probability here because it does not look like there is a strict cutoff point between temperature and a damage incident occurring. The best we can do is ask "At temperature $t$, what is the probability of a damage incident?". The goal of this example is to answer that question.

We need a function of temperature, call it $p(t)$, that is bounded between 0 and 1 (so as to model a probability) and changes from 1 to 0 as we increase temperature. There are actually many such functions, but the most popular choice is the *logistic function.*

$$p(t) = \frac{1}{ 1 + e^{ \;\beta t + \alpha } } $$<br>

$$\alpha \text{ shifts the transition left and right}$$
$$\beta \text{ determines the slope of the transition}$$

<br>
  
  


Our goal is to infer the values of $\alpha$ and $\beta$ given the historical data.  Below is the function plotted for a variety of $\alpha$ and $\beta$ values.

![title](img/logistic.png)

### Setting up the PyMC3 Model
Let's start modeling this in PyMC3. The $\beta, \alpha$ parameters have no reason to be positive, bounded or relatively large, so they are best modeled by a *Normal random variable*, introduced next.

In [3]:
def logistic(x, beta, alpha=0):
    return 1.0 / (1.0 + np.exp(np.dot(beta, x) + alpha))

temperature = challenger_data[:, 0]
D = challenger_data[:, 1]  # defect or not?

with pm.Model() as model:
    beta = pm.Normal("beta", mu=0, tau=0.001, testval=0)
    alpha = pm.Normal("alpha", mu=0, tau=0.001, testval=0)
    p = pm.Deterministic("p", 1.0/(1. + tt.exp(beta*temperature + alpha))) # <== a p for each temp

We have our probabilities, but how do we connect them to our observed data? A *Bernoulli* random variable with parameter $p$, denoted $\text{Ber}(p)$, is a random variable that takes value 1 with probability $p$, and 0 else. Thus, our model can look like:

$$ \text{Defect Incident, $D_i$} \sim \text{Ber}( \;p(t_i)\; ), \;\; i=1..N$$

where $p(t)$ is our logistic function and $t_i$ are the temperatures we have observations about. Notice in the above code we had to set the values of `beta` and `alpha` to 0. The reason for this is that if `beta` and `alpha` are very large, they make `p` equal to 1 or 0. Unfortunately, `pm.Bernoulli` does not like probabilities of exactly 0 or 1, though they are mathematically well-defined probabilities. So by setting the coefficient values to `0`, we set the variable `p` to be a reasonable starting value. This has no effect on our results, nor does it mean we are including any additional information in our prior. It is simply a computational caveat in PyMC3. 

### Connecting the observed data and performing the inference

In [4]:
# connect the probabilities in `p` with our observations through a
# Bernoulli random variable.
with model:
    observed = pm.Bernoulli("bernoulli_obs", p, observed=D)
    
    # Mysterious code to be explained in Chapter 3
    step = pm.Metropolis()
    trace = pm.sample(120000, step=step)
    burned_trace = trace[100000::2]

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [alpha]
>Metropolis: [beta]
100%|██████████| 120500/120500 [00:25<00:00, 4709.15it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


### Examining the posterior distributions of $\alpha$ and $\beta$
We have trained our model on the observed data, now we can sample values from the posterior. Let's look at the posterior distributions for $\alpha$ and $\beta$:

![title](img/alpha_beta.png)

In [6]:
alpha_samples = burned_trace["alpha"][:, None];  # best to make them 1d
beta_samples = burned_trace["beta"][:, None];

All samples of $\beta$ are greater than 0. If instead the posterior was centered around 0, we may suspect that $\beta = 0$, implying that temperature has no effect on the probability of defect. 

Similarly, all $\alpha$ posterior values are negative and far away from 0, implying that it is correct to believe that $\alpha$ is significantly less than 0. 

Regarding the spread of the data, we are very uncertain about what the true parameters might be (though considering the low sample size and the large overlap of defects-to-nondefects this behaviour is perhaps expected).  

### Expected (average) probability vs. temperature

In [7]:
t = np.linspace(temperature.min() - 5, temperature.max()+5, 50)[:, None]

p_t = logistic(t.T, beta_samples, alpha_samples)
mean_prob_t = p_t.mean(axis=0)  # result is a 1D array

![title](img/posterior_p.png)

Above we have averaged over all samples from the posterior to get a likely value for $p(t_i)$. We also plotted two possible realizations of what the actual underlying system might be. Both are equally likely as any other draw. The blue line is what occurs when we average all the 20000 possible dotted lines together.

### Confidence Intervals on Probabilities

An interesting question to ask is for what temperatures are we most uncertain about the defect-probability? Below we plot the expected value line **and** the associated 95% intervals for each temperature. 

![title](img/confidence.png)

The 95% credible interval, or 95% CI, painted in purple, represents the interval, for each temperature, that contains 95% of the distribution. For example, at 65 degrees, we can be 95% sure that the probability of defect lies between 0.25 and 0.75.

### What about the day of the Challenger disaster?

On the day of the Challenger disaster, the outside temperature was 31 degrees Fahrenheit. What is the posterior distribution of a defect occurring,  given this temperature? The distribution is plotted below. **It looks almost guaranteed that the Challenger was going to be subject to defective O-rings.**

![title](img/temperature31.png)

We appear to have "quantified the concerns" with a consistent model that takes advantage of all the data. **Had PyMC3 been available and in use, perhaps the managers would not have dared override the engineers' recommendation not to launch.**